# 🛡️ Hancock v3 — CyberViser Fine-Tune
**Mistral 7B · LoRA (Unsloth) · MITRE ATT&CK + NVD/CVE dataset**

Runs free on: Google Colab (T4) · Kaggle (P100) · SageMaker Studio Lab (T4)

> **Runtime:** Runtime → Change runtime type → **GPU (T4)**

In [ ]:
# ── Step 1: Check GPU ─────────────────────────────────
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

In [ ]:
# ── Step 2: Install dependencies ──────────────────────
!pip install -q 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'
!pip install -q trl>=0.8.0 transformers>=4.40.0 accelerate datasets bitsandbytes peft

In [ ]:
# ── Step 3: Clone Hancock ─────────────────────────────
import os
if not os.path.exists('Hancock'):
    !git clone https://github.com/cyberviser/Hancock.git
%cd Hancock

In [ ]:
# ── Step 4: Set HuggingFace token (optional — for saving model) ───
import os
# Get free token at: https://huggingface.co/settings/tokens
os.environ['HF_TOKEN'] = 'hf_your_token_here'  # ← paste your HF token

In [ ]:
# ── Step 5: Dry run — verify GPU + dataset load ───────
!python hancock_finetune_v3.py --dry-run

In [ ]:
# ── Step 6: TRAIN 🚀 ──────────────────────────────────
# T4/P100 (free): 300 steps ≈ 25-35 min
# Increase --steps for better quality (500 = ~1hr on T4)
!python hancock_finetune_v3.py \
    --steps 300 \
    --push-to-hub \
    --export-gguf

In [ ]:
# ── Step 7: Test the fine-tuned adapter ───────────────
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='hancock-adapter-v3',
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

SYSTEM = 'You are Hancock, an elite cybersecurity specialist built by CyberViser.'
messages = [
    {'role': 'system',  'content': SYSTEM},
    {'role': 'user',    'content': 'Explain CVE-2024-6387 and how to detect exploitation attempts.'},
]
inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to('cuda')
outputs = model.generate(input_ids=inputs, max_new_tokens=512, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# ── Step 8: Download GGUF for local deployment ────────
# After training, download the GGUF file to run Hancock locally
# with llama.cpp, Ollama, or LM Studio — no GPU needed!
from google.colab import files
import glob
gguf = glob.glob('*.gguf')
if gguf:
    print(f'Downloading {gguf[0]}...')
    files.download(gguf[0])
else:
    print('No GGUF found — re-run Step 6 with --export-gguf')